# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 19 2022 9:27AM,242192,12,SB07182022-FL,SugarBear,Released
1,Jul 19 2022 9:22AM,242188,10,PRF-33452,Bio-PRF,Released
2,Jul 19 2022 9:22AM,242188,10,PRF-33456,Bio-PRF,Released
3,Jul 19 2022 9:22AM,242188,10,PRF-33465,Bio-PRF,Released
4,Jul 19 2022 9:22AM,242188,10,PRF-33476,Bio-PRF,Released
5,Jul 19 2022 9:22AM,242188,10,PRF-33480,Bio-PRF,Released
6,Jul 19 2022 9:22AM,242188,10,PRF-33485,Bio-PRF,Released
7,Jul 19 2022 9:22AM,242188,10,PRF-33496,Bio-PRF,Released
8,Jul 19 2022 9:22AM,242188,10,PRF-33512,Bio-PRF,Released
9,Jul 19 2022 9:22AM,242188,10,PRF-33516,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
54,242187,Released,1
55,242188,Released,22
56,242189,Released,1
57,242190,Released,1
58,242192,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242187,NaN,NaN,1.0
242188,NaN,NaN,22.0
242189,NaN,NaN,1.0
242190,NaN,NaN,1.0
242192,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4.0,4.0,4.0
241972,2.0,11.0,1.0
242008,0.0,0.0,1.0
242020,17.0,0.0,2.0
242067,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4,4,4
241972,2,11,1
242008,0,0,1
242020,17,0,2
242067,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,4,4
1,241972,2,11,1
2,242008,0,0,1
3,242020,17,0,2
4,242067,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,4,4
1,241972,2,11,1
2,242008,,,1
3,242020,17,,2
4,242067,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 19 2022 9:27AM,242192,12,SugarBear
1,Jul 19 2022 9:22AM,242188,10,Bio-PRF
23,Jul 19 2022 9:22AM,242190,10,Bio-PRF
24,Jul 19 2022 9:21AM,242185,10,ISDIN Corporation
43,Jul 19 2022 9:19AM,242189,12,Doctors Scientific Organica
44,Jul 19 2022 9:11AM,242187,10,Bio-PRF
45,Jul 19 2022 9:07AM,242184,18,"Blinc International, Inc. - PV"
46,Jul 19 2022 9:07AM,242183,12,Hush Hush
56,Jul 19 2022 9:03AM,242182,10,ISDIN Corporation
93,Jul 19 2022 9:03AM,242181,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 19 2022 9:27AM,242192,12,SugarBear,,,1
1,Jul 19 2022 9:22AM,242188,10,Bio-PRF,,,22
2,Jul 19 2022 9:22AM,242190,10,Bio-PRF,,,1
3,Jul 19 2022 9:21AM,242185,10,ISDIN Corporation,,,19
4,Jul 19 2022 9:19AM,242189,12,Doctors Scientific Organica,,,1
5,Jul 19 2022 9:11AM,242187,10,Bio-PRF,,,1
6,Jul 19 2022 9:07AM,242184,18,"Blinc International, Inc. - PV",,,1
7,Jul 19 2022 9:07AM,242183,12,Hush Hush,,,10
8,Jul 19 2022 9:03AM,242182,10,ISDIN Corporation,,,37
9,Jul 19 2022 9:03AM,242181,10,ISDIN Corporation,,,33


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 9:27AM,242192,12,SugarBear,1,,
1,Jul 19 2022 9:22AM,242188,10,Bio-PRF,22,,
2,Jul 19 2022 9:22AM,242190,10,Bio-PRF,1,,
3,Jul 19 2022 9:21AM,242185,10,ISDIN Corporation,19,,
4,Jul 19 2022 9:19AM,242189,12,Doctors Scientific Organica,1,,
5,Jul 19 2022 9:11AM,242187,10,Bio-PRF,1,,
6,Jul 19 2022 9:07AM,242184,18,"Blinc International, Inc. - PV",1,,
7,Jul 19 2022 9:07AM,242183,12,Hush Hush,10,,
8,Jul 19 2022 9:03AM,242182,10,ISDIN Corporation,37,,
9,Jul 19 2022 9:03AM,242181,10,ISDIN Corporation,33,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 9:27AM,242192,12,SugarBear,1,,
1,Jul 19 2022 9:22AM,242188,10,Bio-PRF,22,,
2,Jul 19 2022 9:22AM,242190,10,Bio-PRF,1,,
3,Jul 19 2022 9:21AM,242185,10,ISDIN Corporation,19,,
4,Jul 19 2022 9:19AM,242189,12,Doctors Scientific Organica,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 9:27AM,242192,12,SugarBear,1.0,NaN,NaN
1,Jul 19 2022 9:22AM,242188,10,Bio-PRF,22.0,NaN,NaN
2,Jul 19 2022 9:22AM,242190,10,Bio-PRF,1.0,NaN,NaN
3,Jul 19 2022 9:21AM,242185,10,ISDIN Corporation,19.0,NaN,NaN
4,Jul 19 2022 9:19AM,242189,12,Doctors Scientific Organica,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 9:27AM,242192,12,SugarBear,1.0,0.0,0.0
1,Jul 19 2022 9:22AM,242188,10,Bio-PRF,22.0,0.0,0.0
2,Jul 19 2022 9:22AM,242190,10,Bio-PRF,1.0,0.0,0.0
3,Jul 19 2022 9:21AM,242185,10,ISDIN Corporation,19.0,0.0,0.0
4,Jul 19 2022 9:19AM,242189,12,Doctors Scientific Organica,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3874499,212.0,13.0,22.0
102,242008,1.0,0.0,0.0
12,726564,12.0,0.0,0.0
18,726535,3.0,0.0,0.0
19,1210765,12.0,4.0,16.0
20,1936441,135.0,26.0,8.0
21,3631740,15.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3874499,212.0,13.0,22.0
1,102,242008,1.0,0.0,0.0
2,12,726564,12.0,0.0,0.0
3,18,726535,3.0,0.0,0.0
4,19,1210765,12.0,4.0,16.0
5,20,1936441,135.0,26.0,8.0
6,21,3631740,15.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,212.0,13.0,22.0
1,102,1.0,0.0,0.0
2,12,12.0,0.0,0.0
3,18,3.0,0.0,0.0
4,19,12.0,4.0,16.0
5,20,135.0,26.0,8.0
6,21,15.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,212.0
1,102,Released,1.0
2,12,Released,12.0
3,18,Released,3.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,18,19,20,21
Status,,,,,,,
Completed,22.0,0.0,0.0,0.0,16.0,8.0,0.0
Executing,13.0,0.0,0.0,0.0,4.0,26.0,0.0
Released,212.0,1.0,12.0,3.0,12.0,135.0,15.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,18,19,20,21
0,Completed,22.0,0.0,0.0,0.0,16.0,8.0,0.0
1,Executing,13.0,0.0,0.0,0.0,4.0,26.0,0.0
2,Released,212.0,1.0,12.0,3.0,12.0,135.0,15.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,18,19,20,21
0,Completed,22.0,0.0,0.0,0.0,16.0,8.0,0.0
1,Executing,13.0,0.0,0.0,0.0,4.0,26.0,0.0
2,Released,212.0,1.0,12.0,3.0,12.0,135.0,15.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()